In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler


import python_code.Scripts as sc
import python_code.Reference as ref



In [2]:
model= pickle.load(open('final_model.sav', 'rb'))

In [3]:

df = pd.read_csv('./data/qc_pub_fy2018.csv')
df.head()

,FSAFIL1,FSAFIL2,FSAFIL3,FSAFIL4,FSAFIL5,FSAFIL6,FSAFIL7,FSAFIL8,FSAFIL9,FSAFIL10,...,WORK12,WORK13,WORK14,WORK15,WORK16,FSNONCIT,FSDIS,FSELDER,FSKID,STATENAME
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,Connecticut
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,Connecticut
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut


In [4]:

df.shape

(43738, 814)

In [5]:

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43738 entries, 0 to 43737
Columns: 814 entries, FSAFIL1 to STATENAME
dtypes: float64(373), int64(440), object(1)
memory usage: 271.6+ MB


In [6]:

df = sc.clean_data_to_predict(df)

AttributeError: module 'python_code.Reference' has no attribute 'corr_features'

In [ ]:

ss= StandardScaler()
X = ss.fit_transform(df.drop(columns=['state']))
df['predict']=model.predict(X)
df

In [ ]:
df.to_csv('./data/2018_indicators/preds_all.csv',index=None)

In [7]:
pred_df = pd.DataFrame(df.groupby('state')['predict'].value_counts().unstack()).rename(columns={0.0:'No',1.0:'Yes'})

KeyError: 'state'

In [8]:
pred_df.info()

NameError: name 'pred_df' is not defined

In [9]:
pred_df.to_csv('./data/2018_indicators/preds_target.csv',index='state')

NameError: name 'pred_df' is not defined

In [10]:

coef_df = pd.read_csv('./data/2018_indicators/coef.csv')
coef_df['diff']= (coef_df['pred_0']-coef_df['pred_1']).abs()
coef_diff = coef_df.drop(columns=['pred_0','pred_1'])

In [11]:
coef_diff = coef_diff.sort_values(by=['diff'],ascending=False).reset_index(drop=True)
top = coef_diff.head(15)
top.to_csv('./data/2018_indicators/top_impacts.csv',index=None)

In [12]:
def top_coef_df(fullname, dataset):
    """Creates a dataframe on .corr(), used for unit categories."""
    fullname_df = pd.DataFrame()
    for ea in fullname['feature']:
        try:
            fullname_df[ea] = dataset[ea]
        except:
            continue
    fullname_df['state']=dataset['state']
    return fullname_df

In [13]:
coef_2018= sc.top_coef_df(top, df).set_index('state')
coef_2018.astype('int64').to_csv('./data/2018_indicators/coef_2018_all.csv',index='state')

KeyError: 'state'

In [14]:
coef_2018.groupby('state').mean().astype('int64').to_csv('./data/2018_indicators/coef_2018_state_avg.csv',index='state')

NameError: name 'coef_2018' is not defined